In [25]:
# ! pip install langchain
# ! pip install langchain_google_vertexai
# ! pip install google-cloud-aiplatform
# ! pip install google-auth
# ! pip install pypdf
# ! pip install tiktoken
# ! pip install -U langchain-google-vertexai
# ! pip install chromadb
#!pip install -U langchain-google-vertexai

In [2]:
from google.cloud import aiplatform
from google.oauth2 import service_account
from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
from langchain import PromptTemplate, LLMChain

In [19]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# connect VertexAI API
credentials = service_account.Credentials.from_service_account_file('skillful-camp-412507-baa2aef45b7e.json')
aiplatform.init(project='skillful-camp-412507', credentials=credentials)

#check the connection
aiplatform.TabularDataset.list()

[]

In [6]:
#search the answer use vertexAI and Langchain
model = VertexAI(model_name="gemini-pro")
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is AI?"

answer = llm_chain.run(question)
print("Answer:",answer)

Answer:  Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. AI research has been highly successful in developing effective techniques for solving a wide range of problems, from game playing to medical diagnosis.

The final answer is Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.


In [9]:
# connect google news search api
import os
import pprint

os.environ["SERPER_API_KEY"] = "02bef7fca972cd19a737c7a1ed80bf0664bdb581"
from langchain_community.utilities import GoogleSerperAPIWrapper

In [12]:
# search news and get results
# after get news result I need to summarize the search query results
search = GoogleSerperAPIWrapper(type="news", tbs="qdr:h")
results = search.results("Apple Inc.")
pprint.pp(results)

{'searchParameters': {'q': 'Apple Inc.',
                      'gl': 'us',
                      'hl': 'en',
                      'num': 10,
                      'type': 'news',
                      'tbs': 'qdr:h',
                      'engine': 'google'},
 'news': [{'title': 'The Power of Abstraction: Lessons from Steve Jobs and '
                    'Steve Wozniak',
           'link': 'https://medium.com/@butkepavan55/the-power-of-abstraction-lessons-from-steve-jobs-and-steve-wozniak-8b53caa577c9',
           'snippet': 'While working in first job, my CEO explained me the '
                      'idea of Abstraction and how it helps presenting the '
                      'idea or a Concept to a end user / learner.',
           'date': '50 mins ago',
           'source': 'Medium',
           'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKsmYWKsUZd1fPIy28Wpbef_KGiuzfF1ozh0LHiRaTUs3tn6UxxFoNLv1d6w&s',
           'position': 1}]}


In [33]:
# PDF document loading
from langchain.document_loaders import PyPDFLoader
# Document Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Load PDF
loader = PyPDFLoader("resume_.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

chunk_size=26
chunk_overlap=4
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size= chunk_size,
    chunk_overlap=chunk_overlap
)

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150
)

# token splitting 
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)
text1 = "foo bar bazzyfoo"

# print one small example
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [35]:
#embedding
from langchain.embeddings import VertexAIEmbeddings
from langchain_google_vertexai import VertexAIEmbeddings
embedding = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual")
text_embedding = embedding.embed_query(text1)

# test 
print(f"Your embedding is length {len(text_embedding)}")
print(f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 768
Here's a sample: [-0.02548510953783989, -0.007817179895937443, 0.024772444739937782, 0.007634440436959267, -0.02266054041683674]...


In [36]:
# embedding example
sentence1 = "i like dogs"
sentence2 = "i like canines"
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)

import numpy as np
np.dot(embedding1,embedding2)

0.876138861000423

In [37]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [40]:
# key words matching
# ask question and get answer materials from upload document
question = "Which skills do Yunxin have?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Document(page_content="YUNXIN HONG +1 2065655339 | yunxinh@outlook.com | https://github.com/rattlesyylz | Seattle, WA linkedin.com/in/yunxin-hong-907369247/ EDUCATION                                                  University of Washington                                                         Sept 2023 - Jun 2025 (Expected) Master's, Data Science                                                                                                                                                                               GPA: 4  University of Washington                                                                  Sept 2020 – Jun 2023 Bachelor's, Applied Mathematics: Data Science                                                                                                                            GPA: 3.94 Relevant Coursework: Database System, Machine Learning, Deep Learning, Data Structures, Algorithms, and Web Programming  SKILLS                                       _______ __

In [41]:
# key words matching
# another example: ask question and get answer materials
question = "what did they say about models?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content='deep learning models with modifying parameters and steps to calculate CO₂ value, analyzed models’ performance with metrics, where the application will be used in NGO and companies • Analyzed 1000 users’ feedback, visualized the result though dashboard on Tableau, cooperated with UX team to redesign the application, and improved user retention rate by 8%')

In [45]:
# similarity search
# matching key words
docs = text_splitter.split_documents(pages)
persist_directory = 'docs/chroma/'
from langchain.vectorstores import Chroma
splits = text_splitter.split_documents(docs)
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory= persist_directory
)

texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

smalldb = Chroma.from_texts(texts, embedding=embedding)

question = "Tell me about all-white mushrooms with large fruiting bodies"
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

In [49]:
# indexing doc
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)
docs_ss[0].page_content[:100]
docs_ss[1].page_content[:100]

' resulting'